# Pregunta 1 – Modelo base (3,0 puntos)

Formula el modelo MILP en GurobiPy y presenta su solución junto con la interpretación de los
resultados para minimizar el coste total de transferencia. Se valorará el uso de una notación
matemática clara y coherente.

In [3]:
import gurobipy as gp
from gurobipy import GRB

In [4]:
# --- 1. DATOS DEL PROBLEMA ---

# Conjuntos de Orígenes (i) y Destinos (j)
Origenes = ['Lisboa', 'Madrid', 'Turin']
Destinos = ['Paris', 'Berlin', 'Varsovia']

# Oferta de cada origen (en MilGb)
Oferta = {
    'Lisboa': 5,
    'Madrid': 6,
    'Turin': 7
}

# Demanda de cada destino (en MilGb)
Demanda = {
    'Paris': 4,
    'Berlin': 5,
    'Varsovia': 9
}

# Costos Unitarios de Transferencia (céntimos/MilGb -> dividido por 100 para obtener €/MilGb)
# c[i, j]
Costo_Unitario_Centimos = {
    ('Lisboa', 'Paris'): 4, ('Lisboa', 'Berlin'): 3, ('Lisboa', 'Varsovia'): 6,
    ('Madrid', 'Paris'): 7, ('Madrid', 'Berlin'): 4, ('Madrid', 'Varsovia'): 9,
    ('Turin', 'Paris'): 9, ('Turin', 'Berlin'): 5, ('Turin', 'Varsovia'): 2
}
Costo_Unitario = {k: v / 100 for k, v in Costo_Unitario_Centimos.items()}

# Parámetros Operativos
COSTO_FIJO_BASE = 50.0  # Costo fijo por activar uno de los 4 canales base
CAPACIDAD_CANAL = 10.0  # Capacidad máxima de flujo por canal (M)
MAX_CANALES_BASE = 4  # Máximo de canales base disponibles

# Parámetros del Canal Adicional
COSTO_FIJO_ADICIONAL = 65.0
MAX_CANAL_ADICIONAL = 1  # A lo sumo puede alquilarse un solo canal adicional

In [5]:
# --- 2. CREACIÓN DEL MODELO ---

m = gp.Model("TransporteConCostosFijos")

# --- 3. VARIABLES DE DECISIÓN ---

# x[i, j]: Flujo de datos (continuas, en MilGb)
x = m.addVars(Origenes, Destinos, vtype=GRB.CONTINUOUS, name="Flujo")

# y[i, j]: Activación del canal (binarias, 1 si se activa, 0 si no)
y = m.addVars(Origenes, Destinos, vtype=GRB.BINARY, name="Activacion")

# z: Alquiler del canal adicional (binaria, 1 si se alquila, 0 si no)
z = m.addVar(vtype=GRB.BINARY, name="CanalAdicional")

Set parameter Username
Set parameter LicenseID to value 2718415
Academic license - for non-commercial use only - expires 2026-10-06


In [6]:
# --- 4. FUNCIÓN OBJETIVO ---
# Minimizar Costo Total = Costo Variable + Costo Fijo Base + Costo Fijo Adicional
Costo_Variable = gp.quicksum(Costo_Unitario[i, j] * x[i, j] for i in Origenes for j in Destinos)
Costo_Fijo_Base = gp.quicksum(COSTO_FIJO_BASE * y[i, j] for i in Origenes for j in Destinos)
Costo_Fijo_Adicional = COSTO_FIJO_ADICIONAL * z

m.setObjective(Costo_Variable + Costo_Fijo_Base + Costo_Fijo_Adicional, GRB.MINIMIZE)

In [7]:
# --- 5. RESTRICCIONES ---

# A. Restricciones de Oferta (Suministro)
m.addConstrs((x.sum(i, '*') == Oferta[i] for i in Origenes), name="Oferta")

# B. Restricciones de Demanda
m.addConstrs((x.sum('*', j) == Demanda[j] for j in Destinos), name="Demanda")

# C. Restricciones de Enlace: Flujo <= Capacidad * Activación (x[i, j] <= M * y[i, j])
# Esto asegura que si hay flujo (x > 0), el canal debe estar activado (y = 1)
m.addConstrs((x[i, j] <= CAPACIDAD_CANAL * y[i, j]
              for i in Origenes for j in Destinos), name="Enlace_Capacidad")

# D. Restricción de Número Máximo de Canales
# El total de canales activados (y) está limitado por los 4 base más el adicional (z)
m.addConstr(y.sum() <= MAX_CANALES_BASE + z, name="MaximoCanales")

# E. Restricción de Exclusión Mutua (Privacidad en Berlín)
# Lisboa-Berlín (y['Lisboa', 'Berlin']) y Madrid-Berlín (y['Madrid', 'Berlin']) no pueden estar activados simultáneamente.
m.addConstr(y['Lisboa', 'Berlin'] + y['Madrid', 'Berlin'] <= 1, name="ExclusionMutua_Berlin")

# F. Restricción de Alquiler Adicional
# z ya está definido como binario, lo que asegura que 'a lo sumo' se alquila uno.

<gurobi.Constr *Awaiting Model Update*>

In [8]:
# --- 6. OPTIMIZAR Y RESOLVER ---
m.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 17 rows, 19 columns and 48 nonzeros
Model fingerprint: 0x722d675d
Variable types: 9 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e-02, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+00]
Presolve time: 0.02s
Presolved: 17 rows, 19 columns, 48 nonzeros
Variable types: 9 continuous, 10 integer (10 binary)
Found heuristic solution: objective 316.0500000
Found heuristic solution: objective 316.0100000

Root relaxation: objective 1.673367e+02, 12 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

  

In [9]:
# --- 7. IMPRIMIR RESULTADOS ---
if m.status == GRB.OPTIMAL:
    print("\n" + "=" * 50)
    print("SOLUCIÓN ÓPTIMA ENCONTRADA")
    print("Costo Total Mínimo: {:.2f} €".format(m.objVal))
    print("=" * 50)

    print("\n--- DETALLE DE FLUJOS (MilGb) ---")
    activados = 0
    for i in Origenes:
        for j in Destinos:
            if x[i, j].x > 1e-6:  # Flujo positivo (mayor que un epsilon)
                print(f"[{i} -> {j}]: {x[i, j].x:.2f} MilGb (Costo variable: {Costo_Unitario[i, j] * x[i, j].x:.2f} €)")
            if y[i, j].x > 0.5:
                activados += 1

    print("\n--- DETALLE DE ACTIVACIÓN DE CANALES ---")
    print(f"Canales Base Activados (y_ij): {activados}")
    print(f"Costo Fijo Base: {activados * COSTO_FIJO_BASE:.2f} €")

    if z.x > 0.5:
        print(f"Canal Adicional Alquilado (z): SÍ")
        print(f"Costo Fijo Adicional: {COSTO_FIJO_ADICIONAL:.2f} €")
    else:
        print(f"Canal Adicional Alquilado (z): NO")
        print(f"Costo Fijo Adicional: 0.00 €")

    print("\nCosto Total (Comprobación):")
    costo_variable_final = sum(Costo_Unitario[i, j] * x[i, j].x for i in Origenes for j in Destinos)
    costo_fijo_final = activados * COSTO_FIJO_BASE + (z.x * COSTO_FIJO_ADICIONAL)
    print(f"  Costo Variable: {costo_variable_final:.2f} €")
    print(f"  Costo Fijo: {costo_fijo_final:.2f} €")
    print(f"  Total: {costo_variable_final + costo_fijo_final:.2f} €")

elif m.status == GRB.INFEASIBLE:
    print("El modelo no tiene solución factible.")



SOLUCIÓN ÓPTIMA ENCONTRADA
Costo Total Mínimo: 200.75 €

--- DETALLE DE FLUJOS (MilGb) ---
[Lisboa -> Berlin]: 5.00 MilGb (Costo variable: 0.15 €)
[Madrid -> Paris]: 4.00 MilGb (Costo variable: 0.28 €)
[Madrid -> Varsovia]: 2.00 MilGb (Costo variable: 0.18 €)
[Turin -> Varsovia]: 7.00 MilGb (Costo variable: 0.14 €)

--- DETALLE DE ACTIVACIÓN DE CANALES ---
Canales Base Activados (y_ij): 4
Costo Fijo Base: 200.00 €
Canal Adicional Alquilado (z): NO
Costo Fijo Adicional: 0.00 €

Costo Total (Comprobación):
  Costo Variable: 0.75 €
  Costo Fijo: 200.00 €
  Total: 200.75 €
